In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_part_number)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for part_number in list_part_number:
    work.put_nowait(part_number)

def crawler():
    global df
    
    while not work.empty():
        part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get('https://www.mevotech.com/part/' + part_number,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = []
                list_tr = html.xpath('//div[@class="atwm-table-continer"]/table/tbody/tr')
                for tr in list_tr:
                    vehicle_temp = tr.xpath('./td/text()')[0].strip() + ' '
                    vehicle_temp += tr.xpath('./td/text()')[1].strip() + ' '
                    if '-' in tr.xpath('./td/text()')[2]:
                        vehicle_temp += tr.xpath('./td/text()')[2].split('-')[1].strip() + '-' + tr.xpath('./td/text()')[2].split('-')[0].strip() + ' '
                    else:
                        vehicle_temp += tr.xpath('./td/text()')[2].strip() + ' '
                    vehicle_temp += tr.xpath('./td/text()')[3].strip()
                    list_vehicle.append(vehicle_temp)

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Number': part_number,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Number': part_number}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：5234

CMK5313  <->  [ok] - 剩余数量：5229
CMK100003  <->  [ok] - 剩余数量：5228
CMK3198  <->  [ok] - 剩余数量：5227
CMK5311  <->  [ok] - 剩余数量：5226


KeyboardInterrupt
2023-12-07T09:47:12Z


KeyboardInterrupt: 